In [ ]:
import sys; sys.path.append("../../../../..")

import torch
import numpy as np

from src.experiment import AttentionExperiment, ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params
from src.utils.attention_utils import reduce_attention_dist
from src.utils.classification_utils import run_bootstrapping

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
params = Params.read_params("gru-params.json")
print("layers = {}".format(params.intermediary_task["attention"]["layers"]))
print("reducer = {}".format(params.intermediary_task["attention"]["reducer"]))
print("n_components = {}".format(params.intermediary_task["attention"]["n_components"]))
print("dropout = {}".format(params.final_task["dropout"]))
print("hidden_dim = {}".format(params.final_task["hidden_dim"]))

In [ ]:
dataset = ExperimentDataset.init_dataset(params.dataset)
dataset

In [ ]:
attention_dataloader = dataset.return_dataloader() 
attention_experiment = AttentionExperiment.initialize_attention_experiment(
    params.intermediary_task, 
    params.dataset,
    from_pretrained=True,
    verbose=True
)

In [ ]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

In [ ]:
lengths = [int(d["pre_lens"].numpy()) for d in dataset]
reduced_attention = reduce_attention_dist(attention_scores, params.intermediary_task["attention"], lengths)
dataset.add_data(reduced_attention, "attention_dist")
print("reduced_attention.shape = {}".format(reduced_attention.shape))

### This is where the classification experiment starts

We create a classification experiment that contains useful methods for classifying bias based on the attention distributions. 

In [ ]:
classification_experiment = ClassificationExperiment.init_cls_experiment(
    params.final_task, 
    params.intermediary_task["attention"]
)

In [ ]:
stats = run_bootstrapping(
    classification_experiment, 
    dataset, 
    params.final_task, 
    num_bootstrap_iters=20,
    input_key="attention_dist", 
    label_key="bias_label",
    shuffle_data=True,
    seed=0
)

In [ ]:
stats

In [ ]:
gru_attention_dataloader = dataset.return_dataloader() 
batch_attention_scores = classification_experiment.extract_attention_scores(
    gru_attention_dataloader,
    input_key="attention_dist", 
    label_key="bias_label",
    seq_len_key="pre_lens",
    attention_mask_key="masks"
)
attention_scores = np.array([score.numpy() for batch in batch_attention_scores for score in batch])

In [ ]:
from scipy.stats import entropy

labels = dataset.get_val('bias_label')
labels_0_indices = (labels == 0).nonzero()
labels_1_indices = labels.nonzero()

attention_scores_0 = attention_scores[labels_0_indices].squeeze() # epistemological 
attention_scores_1 = attention_scores[labels_1_indices].squeeze() # framing

entropy_0 = entropy(attention_scores_0.T)
entropy_1 = entropy(attention_scores_1.T)

avg_entropy_0 = np.mean(entropy_0)
avg_entropy_1 = np.mean(entropy_1)

print("Entropy Epistemological: {} \t Entropy Framing {}".format(avg_entropy_0, avg_entropy_1))